In [1]:
!nvidia-smi

Tue Sep 28 20:45:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   31C    P8     9W / 250W |     19MiB / 11176MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:06:00.0 Off |                  N/A |
| 23%   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
from deeplab import DeepLabV3Plus

Using TensorFlow backend.


In [3]:
deeplabv3plus = DeepLabV3Plus(img_height=512, img_width=800, nclasses=3)

*** Building DeepLabv3Plus Network ***













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(?, 16, 25, 256)

*** Output_Shape => (None, 512, 800, 3) ***


In [4]:
deeplabv3plus.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 800, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 806, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 400, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 256, 400, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [5]:
import glob
import re
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
import numpy as np

def sorted_nicely( li ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(li, key = alphanum_key)
    
orig_list = glob.glob('/home/sami/Downloads/Canola Dataset/labeled/Final 98/Aug_xtrain/split/*.png')
orig_list = sorted_nicely(orig_list)
anot_list = glob.glob('/home/sami/Downloads/Canola Dataset/labeled/Final 98/Aug_ytrain/ysplit/*.png')
anot_list = sorted_nicely(anot_list)
orig_train, orig_test, anot_train, anot_test = train_test_split(orig_list,anot_list,test_size = 0.1)

x_train = np.asarray(orig_train)
y_train = np.asarray(anot_train)
x_val   = np.asarray(orig_test)
y_val   = np.asarray(anot_test)

Y_train = np.array([image.img_to_array(image.load_img(fname)) for fname in y_train])
X_train = np.array([image.img_to_array(image.load_img(fname)) for fname in x_train]) #
Y_val = np.array([image.img_to_array(image.load_img(fname)) for fname in y_val])
X_val = np.array([image.img_to_array(image.load_img(fname)) for fname in x_val])
"""
x_train = np.asarray(orig_list)
y_train = np.asarray(anot_list)
Y_train = np.array([image.img_to_array(image.load_img(fname,target_size = (576,768))) for fname in y_train])

X_train = np.array([image.img_to_array(image.load_img(fname,target_size = (576,768))) for fname in x_train])"""

'\nx_train = np.asarray(orig_list)\ny_train = np.asarray(anot_list)\nY_train = np.array([image.img_to_array(image.load_img(fname,target_size = (576,768))) for fname in y_train])\n\nX_train = np.array([image.img_to_array(image.load_img(fname,target_size = (576,768))) for fname in x_train])'

In [6]:
X_train.shape

(5976, 512, 800, 3)

In [5]:
import tensorflow as tf
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
    denominator = tf.reduce_sum(y_true + y_pred, axis=-1)

    return 1 - (numerator + 1) / (denominator + 1)

In [6]:
from keras.optimizers import Adam
opt = Adam(lr=5e-4)
deeplabv3plus.compile(loss=dice_loss, optimizer=opt, metrics=["accuracy"])


In [13]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
callbacks = [
    EarlyStopping(patience=20, verbose=1),
    ModelCheckpoint('deeplabv3+withresnet50new.h5', verbose=1, save_best_only=True, save_weights_only=True)
]


In [11]:
#deeplabv3plus.load_weights('deeplabv3+withresnet50new.h5')

In [14]:
history = deeplabv3plus.fit(X_train, copied, batch_size=3, epochs=300,callbacks=callbacks,validation_data=(X_val,val))


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 5976 samples, validate on 664 samples
Epoch 1/300





5976/5976 [==============================] - 1170s 196ms/step - loss: 0.0664 - acc: 0.9035 - val_loss: 0.0559 - val_acc: 0.9163
Epoch 2/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0436 - acc: 0.9354 - val_loss: 0.0488 - val_acc: 0.9298
Epoch 3/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0416 - acc: 0.9381 - val_loss: 0.1176 - val_acc: 0.8234
Epoch 4/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0361 - acc: 0.9462 - val_loss: 0.0408 - val_acc: 0.9394
Epoch 5/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0344 - acc: 0.9487 - val_loss: 0.0381 - val_acc: 0.9432
Epoch 6/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0315 - acc: 0.9529 - val_loss: 0.3232 - val

5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0204 - acc: 0.9694 - val_loss: 0.0223 - val_acc: 0.9666
Epoch 53/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0198 - acc: 0.9704 - val_loss: 0.0227 - val_acc: 0.9660
Epoch 54/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0196 - acc: 0.9706 - val_loss: 0.0245 - val_acc: 0.9633
Epoch 55/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0196 - acc: 0.9706 - val_loss: 0.0225 - val_acc: 0.9663
Epoch 56/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0197 - acc: 0.9705 - val_loss: 0.0217 - val_acc: 0.9674
Epoch 57/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0197 - acc: 0.9706 - val_loss: 0.0226 - val_acc: 0.9662
Epoch 58/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0194 - acc: 0.9710 - val_loss: 0.0216 - val_acc: 0.9676
Epoch 59/300
5976/5976 [==

Epoch 114/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0170 - acc: 0.9745 - val_loss: 0.0203 - val_acc: 0.9696
Epoch 115/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0171 - acc: 0.9744 - val_loss: 0.0209 - val_acc: 0.9687
Epoch 116/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0170 - acc: 0.9746 - val_loss: 0.0206 - val_acc: 0.9691
Epoch 117/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0169 - acc: 0.9746 - val_loss: 0.0248 - val_acc: 0.9628
Epoch 118/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0170 - acc: 0.9746 - val_loss: 0.0230 - val_acc: 0.9655
Epoch 119/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0169 - acc: 0.9747 - val_loss: 0.0211 - val_acc: 0.9684
Epoch 120/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0169 - acc: 0.9746 - val_loss: 0.0205 - val_acc: 0.9693
Epoch 

Epoch 145/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0162 - acc: 0.9756 - val_loss: 0.0218 - val_acc: 0.9674
Epoch 146/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0162 - acc: 0.9758 - val_loss: 0.0207 - val_acc: 0.9690
Epoch 147/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0162 - acc: 0.9757 - val_loss: 0.0194 - val_acc: 0.9709
Epoch 148/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0161 - acc: 0.9758 - val_loss: 0.0205 - val_acc: 0.9693
Epoch 149/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0168 - acc: 0.9748 - val_loss: 0.0205 - val_acc: 0.9693
Epoch 150/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0160 - acc: 0.9761 - val_loss: 0.0201 - val_acc: 0.9699
Epoch 151/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0161 - acc: 0.9759 - val_loss: 0.0203 - val_acc: 0.9696
Epoch 

Epoch 176/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0155 - acc: 0.9768 - val_loss: 0.0221 - val_acc: 0.9669
Epoch 177/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0154 - acc: 0.9769 - val_loss: 0.0205 - val_acc: 0.9692
Epoch 178/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0155 - acc: 0.9768 - val_loss: 0.0207 - val_acc: 0.9690
Epoch 179/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0154 - acc: 0.9769 - val_loss: 0.0200 - val_acc: 0.9700
Epoch 180/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0153 - acc: 0.9770 - val_loss: 0.0194 - val_acc: 0.9710
Epoch 181/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0153 - acc: 0.9770 - val_loss: 0.0195 - val_acc: 0.9708
Epoch 182/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0156 - acc: 0.9767 - val_loss: 0.0247 - val_acc: 0.9630
Epoch 

Epoch 207/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0147 - acc: 0.9779 - val_loss: 0.0198 - val_acc: 0.9704
Epoch 208/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0148 - acc: 0.9779 - val_loss: 0.0198 - val_acc: 0.9704
Epoch 209/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0147 - acc: 0.9780 - val_loss: 0.0195 - val_acc: 0.9708
Epoch 210/300
5976/5976 [==============================] - 1106s 185ms/step - loss: 0.0150 - acc: 0.9775 - val_loss: 0.0194 - val_acc: 0.9709
Epoch 211/300
5976/5976 [==============================] - 1106s 185ms/step - loss: 0.0147 - acc: 0.9780 - val_loss: 0.0190 - val_acc: 0.9715
Epoch 212/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0146 - acc: 0.9781 - val_loss: 0.0189 - val_acc: 0.9716
Epoch 213/300
5976/5976 [==============================] - 1106s 185ms/step - loss: 0.0148 - acc: 0.9778 - val_loss: 0.0192 - val_acc: 0.9712
Epoch 

Epoch 238/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0142 - acc: 0.9788 - val_loss: 0.0188 - val_acc: 0.9718
Epoch 239/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0141 - acc: 0.9788 - val_loss: 0.0208 - val_acc: 0.9689
Epoch 240/300
5976/5976 [==============================] - 1104s 185ms/step - loss: 0.0141 - acc: 0.9789 - val_loss: 0.0189 - val_acc: 0.9716
Epoch 241/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0141 - acc: 0.9789 - val_loss: 0.0199 - val_acc: 0.9701
Epoch 242/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0141 - acc: 0.9789 - val_loss: 0.0211 - val_acc: 0.9684
Epoch 243/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0141 - acc: 0.9789 - val_loss: 0.0187 - val_acc: 0.9719
Epoch 244/300
5976/5976 [==============================] - 1105s 185ms/step - loss: 0.0145 - acc: 0.9783 - val_loss: 0.0192 - val_acc: 0.9712
Epoch 

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
print(deeplabv3plus.output.shape)

In [13]:
def rescale_frame(frame, percent=75):
    import cv2
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)


def evaluation(y,stage,model_name,pred,pred2):#,pred2
  
    pred1=(pred+1.0)/2.0
    gt = (y+1.0)/2.0
    pr=pred1.reshape((pred1.shape[0],384,  512, 3)).argmax(axis=3)
    tp = np.zeros(3)
    fp = np.zeros(3)
    fn = np.zeros(3)

    n_pixels = np.zeros(3)
    gt = gt.argmax(-1)
    pr = pr.flatten()
    gt = gt.flatten()
    for cl_i in range(3):
        tp[ cl_i ] += np.sum( (pr == cl_i) * (gt == cl_i) )
        fp[ cl_i ] += np.sum( (pr == cl_i) * ((gt != cl_i)) )
        fn[ cl_i ] += np.sum( (pr != cl_i) * ((gt == cl_i)) )
        n_pixels[ cl_i ] += np.sum( gt == cl_i  )

    cl_wise_score = tp / ( tp + fp + fn + 0.000000000001 )
    n_pixels_norm = n_pixels /  np.sum(n_pixels)
    frequency_weighted_IU = np.sum(cl_wise_score*n_pixels_norm)
    mean_IU = np.mean(cl_wise_score)
    dice = 2*tp/(2*tp+fp+fn)
    print(dice,'Dice class wise')
    print(np.mean(dice),'mean Dice')
    print("frequency_weighted_IU",frequency_weighted_IU , "mean_IU",mean_IU , "class_wise_IU",cl_wise_score)

    return

In [15]:
deeplabv3plus.load_weights('deeplabv3+withresnet50.h5')
